# Essential functionality



In [1]:
import pandas as pd
import numpy as np

## Reindexing

reindex means to create a new object with the data confirmed to a new index

In [2]:
obj = pd.Series([4.5,7.2,-5.3,3.6], index=['d','b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [3]:
obj2 = obj.reindex(['a','b','c','d','e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [4]:
# for ordered data like time series it may be desirable to do some interpolation or filling of values when reindexing
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0,2,4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [5]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [7]:
# reindex can altar either the row/index, or columns, or both.
frame = pd.DataFrame(np.arange(9).reshape((3,3)),index=['a','c','d'], columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [8]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


### Dropping entities from an Axis

In [9]:
obj = pd.Series(np.arange(5.), index=['a','b','c','d','e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [10]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [11]:
obj.drop(['c','d'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [13]:
# Index values can be deleted from either axis

data = pd.DataFrame(np.arange(16).reshape((4,4)), 
    index=['Ohio','Colorado','Utah','New York'], 
    columns=['one','two','three','four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [14]:
# calling drop with a sequence of labels will drop values from the row lables (axis 0)
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [15]:
# you can drop values from the columns by passing axis=1 or axis='columns'
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [16]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


## Indexing, Selection and Filtering

In [18]:
obj = pd.Series(np.arange(4.), index=['a','b','c','d'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [19]:
obj['b']

1.0

In [20]:
obj[1]

1.0

In [21]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [22]:
obj[['b','a','d']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [23]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [24]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [25]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [26]:
# setting using these methods modifies th corresponding section of the Series
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [27]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [28]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [29]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [30]:
data[data['three'] > 5] # selects only the rows where col theree is lager than 5

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [31]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [33]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selection with loc and iloc

loc and iloc enable you to select a subset of the rows and columns from a DataFrame

In [34]:
data.loc['Colorado', ['two','three']]

two      5
three    6
Name: Colorado, dtype: int64

In [35]:
data.iloc[2,[3,0,1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [36]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [37]:
data.iloc[[1,2],[3,0,1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [39]:
# both indexing labels work with slices in addition to single lables or lists of labels
data.loc[:'Utah', 'two']


Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int64

In [40]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


### Integer Indexes
